### Walking & Health Notebook

#### Part I. Data Importation and Cleaning

In [1]:
import pandas as pd
import numpy as np


In [5]:
walkability = pd.read_csv("epa_walkability.csv")
health = pd.read_csv("places_health.csv")
relation = pd.read_csv("zcta_tract_relationship.txt", sep = "|")

/Users/jonathanyun/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3098: DtypeWarning: Columns (8,9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
